In [1]:
import os
from utils import utils
import pandas as pd
import torch
from datetime import datetime
from collections import Counter
from sklearn.metrics import f1_score, recall_score, precision_score
import numpy as np
from transformers.trainer_utils import set_seed
from transformers import RobertaTokenizer
from transformers import AutoConfig, AutoModelWithHeads
from transformers.adapters.composition import Stack
from transformers import TrainingArguments, EvalPrediction, AdapterTrainer
from utils.evaluation import compute_pearsonr


dateTimeObj = datetime.now()

RANDOM_SEED = 42
set_seed(RANDOM_SEED)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Data

In [2]:
""" load data """

train_data, val_data, test_data = utils.load_wassa_dataset()

In [3]:
""" get task labels """

prediction_task = 'empathy'

train_labels = list(train_data[prediction_task].values)
val_labels = list(val_data[prediction_task].values)
test_labels = list(test_data[prediction_task].values)

In [4]:
""" Prepare dataset for training: feature encodings """

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_encodings = tokenizer(list(train_data['essay'].values), truncation=True, padding=True)
val_encodings = tokenizer(list(val_data['essay'].values), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data['essay'].values), truncation=True, padding=True)

In [5]:
""" setup torch dataset """

class WassaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = WassaDataset(train_encodings, train_labels)
val_dataset = WassaDataset(val_encodings, val_labels)
test_dataset = WassaDataset(test_encodings, test_labels)

# Stacking Emotion Prediction

In [6]:
""" init model """

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1,
    hidden_dropout_prob=.01,
)
model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/adapters/models/roberta.py:250: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/adapters/models/roberta.py:228: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a B

In [7]:
""" load emotion adapter, using: https://huggingface.co/AdapterHub/roberta-base-pf-emotion """

emotion_adapter = model.load_adapter('AdapterHub/roberta-base-pf-emotion', source="hf")

In [8]:
""" add adapter for emotion prediction task """

adapter_name = "EMP_emotion_stack" if prediction_task == 'empathy' else 'DIS_emotion_stack'

model.add_adapter(adapter_name)
model.add_classification_head(adapter_name, num_labels=1)

In [9]:
""" activate adapter stack """

model.active_adapters = Stack(emotion_adapter, adapter_name)

In [10]:
""" activate adapter for training """

model.train_adapter([adapter_name])

In [11]:
""" training arguments """

approach = 'emotion-stack'
training_output_dir = f"./training_output/{approach}/{prediction_task}_{dateTimeObj.hour}{dateTimeObj.minute}-{dateTimeObj.day}-{dateTimeObj.month}"
num_train_epochs=20
per_device_train_batch_size=8
per_device_eval_batch_size=8
metric_for_best_model='eval_pearsonr'
warmup_steps=1000
weight_decay=0.1
learning_rate=1e-04


training_args = TrainingArguments(
    seed=RANDOM_SEED,
    output_dir=training_output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    learning_rate=learning_rate,
     warmup_steps=warmup_steps,
     weight_decay=weight_decay,
    logging_dir='./logs',
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
    evaluation_strategy='steps',
    disable_tqdm=False,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    save_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model=metric_for_best_model,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
""" setup trainer """

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_pearsonr,
)

In [13]:
""" train """

trainer.train()

/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1860
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4660


Step,Training Loss,Validation Loss,Pearsonr,Pearsonr Scipy,Pval
50,19.309900,16.890991,0.032400,0.032386,0.596227
100,16.915300,14.795216,-0.025300,-0.025309,0.678867
150,12.537700,9.282456,-0.046500,-0.046473,0.446959
200,5.498100,3.572055,0.041800,0.041799,0.494016
250,3.822200,3.515375,0.128600,0.128594,0.034688
300,4.415000,3.281997,0.232700,0.232664,0.000114
350,2.996200,3.309586,0.288500,0.288478,0.000001
400,3.115500,3.248243,0.366900,0.366937,0.000000
450,2.901800,2.992455,0.402500,0.402491,0.000000
500,3.018400,2.980379,0.405900,0.405915,0.000000


***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
Saving model checkpoint to ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/emotion/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/emotion/pytorch_adapter.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-50/EMP_emotion_stack/adapter_config.json
Module weights saved i

Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-200/EMP_emotion_stack/head_config.json
Module w

***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
Saving model checkpoint to ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/emotion/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/emotion/pytorch_adapter.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-400/EMP_emotion_stack/adapter_config.json
Module weights

...

Saving model checkpoint to ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/emotion/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/emotion/pytorch_adapter.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4050/EMP_emotion_stack/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/c

Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4200/EMP_emotion_stack/head_config.json


***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
Saving model checkpoint to ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/emotion/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/emotion/pytorch_adapter.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4400/EMP_emotion_stack/adapter_config.json
Module

Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/EMP_emotion_stack/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/EMP_emotion_stack/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_1921-20-4/checkpoint-4550/emotion/pytorch_model_head.bin


TrainOutput(global_step=4660, training_loss=1.98271719723812, metrics={'train_runtime': 226.646, 'train_samples_per_second': 164.133, 'train_steps_per_second': 20.561, 'total_flos': 4371485723748000.0, 'train_loss': 1.98271719723812, 'epoch': 20.0})

# Evaluate

In [14]:
""" output eval metrics from best model """

trainer.model.cuda()
eval_output = trainer.evaluate()
eval_result = eval_output[metric_for_best_model]

pd.DataFrame({'metric':list(eval_output.keys()), 'value': list(eval_output.values())}, columns=['metric', 'value'])

***** Running Evaluation *****
  Num examples = 270
  Batch size = 8


,metric,value
0,eval_loss,3.289504e+00
1,eval_pearsonr,4.754000e-01
2,eval_pearsonr_scipy,4.754218e-01
3,eval_pval,1.242750e-16
4,eval_runtime,4.421000e-01
5,eval_samples_per_second,6.107850e+02
6,eval_steps_per_second,7.691400e+01
7,epoch,2.000000e+01


# Test predictions

In [15]:
""" make predictions on test dataset """

p = trainer.predict(test_dataset)
preds = p.predictions[:, 0]


***** Running Prediction *****
  Num examples = 525
  Batch size = 8
/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [16]:
""" save predictions """

pred_file = f"{approach}-{metric_for_best_model}-{round(eval_result * 100, 4)}_{dateTimeObj.hour}{dateTimeObj.minute}-{dateTimeObj.day}-{dateTimeObj.month}.tsv"
pred_path = f'./predictions/{prediction_task}/{pred_file}'
os.makedirs(f'./predictions/{prediction_task}/', exist_ok=True)
pd.Series(preds).to_csv(pred_path, sep='\t', header=False, index=False)
print("saved predictions to",pred_path)

saved predictions to ./predictions/empathy/emotion-stack-eval_pearsonr-47.54_1921-20-4.tsv


# Save the adapter

In [17]:
# trainer.model.save_adapter(f"./trained_adapters/{adapter_name}", adapter_name)